# Chapter 8 Toolkit — Pattern Recognition + VC Theory 🧠📊

## Welcome to Pattern Recognition and Statistical Learning Theory!

This notebook is your comprehensive guide to understanding **how machines learn to classify data** and **why we can trust their predictions**. We'll take you step-by-step through the foundational concepts that power modern machine learning.

### 📚 What You'll Learn (in order):

1. **Foundation: Risk and Loss** 
   - What does it mean for a classifier to make "good" predictions?
   - How do we measure classification error?

2. **Linear Classifiers (The Perceptron)** 
   - The simplest learning algorithm that works!
   - When does it succeed? When does it fail?

3. **Kernels: Escaping Linear Limitations** 
   - What if data isn't linearly separable?
   - The "kernel trick" that transforms your data

4. **Empirical Risk Minimization (ERM)** 
   - The fundamental principle: minimize training error
   - Why training error ≠ true error (and how to fix it)

5. **VC Theory: The Mathematics of Generalization** 
   - Why do machine learning models work on unseen data?
   - Understanding model complexity and overfitting
   - Rigorous probability bounds for generalization

6. **Concentration Inequalities** 
   - Statistical guarantees for finite samples
   - How much data do you need to trust your model?

### 🎯 Why This Matters:

Pattern recognition is about **learning from examples**. Given data points with labels (e.g., spam/not spam, cat/dog), we want to:
- Build a **classifier** that makes accurate predictions
- Understand **when and why** it works
- Get **mathematical guarantees** about future performance

This isn't just theory—these concepts underpin all of supervised learning, from neural networks to support vector machines!

### 💡 How to Use This Notebook:

1. **First time**: Run cells **top-to-bottom** to see concepts build upon each other
2. **For review**: Jump to specific sections using the structure above
3. **For exams**: Copy functions from the "Function Index" at the bottom
4. **For understanding**: Read the explanations slowly—we've made them as clear as possible!

Let's begin! 🚀

In [ ]:
import numpy as np
import math
from dataclasses import dataclass
from typing import Callable, Dict, Tuple, Optional, List

import matplotlib.pyplot as plt


## 1. Foundation: Understanding Risk and Loss 🎯

### What is Classification?

Imagine you have data points $X$ (features) and labels $Y$ (categories). For example:
- Emails ($X$) labeled as spam or not spam ($Y$)
- Images ($X$) labeled as cat or dog ($Y$)
- Patients ($X$) labeled as healthy or sick ($Y$)

**Goal**: Build a function $g(X)$ that predicts $Y$ as accurately as possible.

### The 0-1 Loss: The Most Natural Error Measure

The **0-1 loss** simply counts mistakes:
$$\ell(y, \hat{y}) = \begin{cases} 0 & \text{if } y = \hat{y} \text{ (correct)} \\ 1 & \text{if } y \neq \hat{y} \text{ (wrong)} \end{cases}$$

**Why 0-1 loss?** It directly measures what we care about: did we get it right or wrong?

### Two Types of Risk: True vs. Empirical

#### 1. **True Risk** $R(g)$ — The Ideal (but unknown)

$$R(g) = \mathbb{E}[\mathbf{1}\{Y \neq g(X)\}] = \mathbb{P}(Y \neq g(X))$$

This is the **probability** that classifier $g$ makes a mistake on a **random new example**. 

**Problem**: We can't compute this! We don't know the true distribution of $(X, Y)$.

#### 2. **Empirical Risk** $\hat{R}_n(g)$ — What We Can Actually Compute

$$\hat{R}_n(g) = \frac{1}{n}\sum_{i=1}^n \mathbf{1}\{Y_i \neq g(X_i)\}$$

This is just the **fraction of training examples** that $g$ gets wrong. We use $\hat{R}_n(g)$ as an **estimate** of $R(g)$.

### Key Insight: The Central Question of Machine Learning

$$\boxed{\text{Training error } \hat{R}_n(g) \approx \text{ True error } R(g) \text{ ?}}$$

**When is this approximation good?**
- With **enough data** ($n$ large)
- With **simple models** (not too many parameters)
- With **independent test data** (never used for training!)

This entire chapter is about making this approximation rigorous!

### Why These Utilities Matter

Below are reusable functions for:
- Converting between label formats ({0,1} ↔ {-1,+1})
- Computing 0-1 loss and empirical risk
- Calculating confusion matrices, precision, recall, F1-score

In [ ]:
def to_pm1(y: np.ndarray) -> np.ndarray:
    """Convert labels in {0,1} to {-1,+1}. If already {-1,+1}, keep."""
    y = np.asarray(y).astype(int)
    vals = set(np.unique(y).tolist())
    if vals.issubset({-1, 1}):
        return y
    if vals.issubset({0, 1}):
        return 2*y - 1
    raise ValueError("y must be in {0,1} or {-1,1}.")

def to_01(y_pm1: np.ndarray) -> np.ndarray:
    """Convert labels in {-1,+1} to {0,1}."""
    y = np.asarray(y_pm1).astype(int)
    vals = set(np.unique(y).tolist())
    if not vals.issubset({-1, 1}):
        raise ValueError("y must be in {-1,1}.")
    return (y + 1)//2

def zero_one_loss(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)
    return (y_true != y_pred).astype(float)

def empirical_risk(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.mean(zero_one_loss(y_true, y_pred)))

@dataclass
class Confusion:
    tp: int; fp: int; tn: int; fn: int

def confusion_matrix_01(y_true01: np.ndarray, y_pred01: np.ndarray) -> Confusion:
    y_true01 = np.asarray(y_true01).astype(int)
    y_pred01 = np.asarray(y_pred01).astype(int)
    tp = int(np.sum((y_true01==1) & (y_pred01==1)))
    fp = int(np.sum((y_true01==0) & (y_pred01==1)))
    tn = int(np.sum((y_true01==0) & (y_pred01==0)))
    fn = int(np.sum((y_true01==1) & (y_pred01==0)))
    return Confusion(tp=tp, fp=fp, tn=tn, fn=fn)

def precision_recall_f1(y_true01: np.ndarray, y_pred01: np.ndarray) -> Dict[str, float]:
    C = confusion_matrix_01(y_true01, y_pred01)
    precision = C.tp / (C.tp + C.fp) if (C.tp + C.fp) > 0 else float("nan")
    recall = C.tp / (C.tp + C.fn) if (C.tp + C.fn) > 0 else float("nan")
    f1 = 2*precision*recall/(precision+recall) if (precision==precision and recall==recall and (precision+recall)>0) else float("nan")
    acc = (C.tp + C.tn) / max(1, (C.tp + C.fp + C.tn + C.fn))
    return {"accuracy": float(acc), "precision": float(precision), "recall": float(recall), "f1": float(f1), **C.__dict__}


### 📊 Test Set Guarantee: Hoeffding's Inequality

#### The Problem with Training Error

If you train a model on data and measure its error **on the same data**, you get an **optimistically biased** estimate. The model has "seen" this data, so it performs artificially well.

#### The Solution: An Independent Test Set

**Key Idea**: Split your data into:
- **Training set** (size $n$): Used to learn the model $\hat{g}$
- **Test set** (size $m$): **Never** shown to the model during training

Now measure $\hat{R}_m(\hat{g})$ on the test set. This is an unbiased estimate of $R(\hat{g})$!

#### Hoeffding's Bound: How Good is Our Test Estimate?

For a **fixed** classifier $\hat{g}$ and **independent** test set of size $m$:

$$\boxed{\mathbb{P}\Big(|\hat{R}_m(\hat{g}) - R(\hat{g})| > \varepsilon\Big) \leq 2e^{-2m\varepsilon^2}}$$

**What this means**:
- With high probability (say 95%), $\hat{R}_m$ is close to $R$
- **More test data** ($m$ large) → **tighter** guarantee
- **Smaller tolerance** ($\varepsilon$ small) → need more data

**Example**: For $\varepsilon = 0.05$ and confidence $\delta = 0.05$, you need $m \approx 738$ test examples.

#### Why This is Important

This bound tells you:
1. How much to trust your test error
2. How many test examples you need for a reliable estimate
3. The trade-off between data size and accuracy of estimation

In [ ]:
def hoehffding_test_bound(m: int, eps: float) -> float:
    """2 * exp(-2 m eps^2) for bounded loss in [0,1]."""
    if m <= 0 or eps <= 0:
        raise ValueError("m>0 and eps>0 required.")
    return float(2 * math.exp(-2 * m * eps * eps))

def required_test_size_for_eps_delta(eps: float, delta: float) -> int:
    """Small helper: choose m so that 2 exp(-2 m eps^2) <= delta."""
    if not (0 < delta < 1) or eps <= 0:
        raise ValueError("delta in (0,1), eps>0.")
    m = math.log(2/delta) / (2 * eps * eps)
    return int(math.ceil(m))

# Example:
eps, delta = 0.03, 0.05
print("m needed for eps=0.03, delta=0.05:", required_test_size_for_eps_delta(eps, delta))
print("bound check:", hoehffding_test_bound(required_test_size_for_eps_delta(eps, delta), eps))


## 2. Linear Classifiers: The Perceptron Algorithm 🧠

### What is a Linear Classifier?

Imagine drawing a **line** (in 2D) or **hyperplane** (in higher dimensions) that separates two classes:

```
    Class +1          |          Class -1
      ●  ●            |            ○  ○
        ●             |          ○
      ●               |            ○
  ──────────────────────────────────────
           ↑ This line is our classifier
```

**Mathematical form**: 
$$g(x) = \text{sign}(w^\top x + b)$$

where:
- $w$ is the **weight vector** (perpendicular to the line)
- $b$ is the **bias** (shifts the line)
- $\text{sign}(z) = \begin{cases} +1 & \text{if } z \geq 0 \\ -1 & \text{if } z < 0 \end{cases}$

### The Perceptron: Learning the Line

**Perceptron** is an algorithm that learns $w$ and $b$ from labeled examples.

#### How It Works (Intuition):

1. **Start** with random (or zero) weights
2. **Loop** through training examples:
   - If example $i$ is **correctly classified**: do nothing
   - If example $i$ is **misclassified**: adjust $w$ and $b$ to move toward the correct side
3. **Repeat** until no mistakes (or max iterations reached)

#### The Update Rule:

When point $(x_i, y_i)$ is misclassified (i.e., $y_i(w^\top x_i + b) \leq 0$):

$$w \leftarrow w + y_i x_i$$
$$b \leftarrow b + y_i$$

**Intuition**: We "push" the decision boundary in the direction that would correctly classify $x_i$.

### When Does Perceptron Work?

**Theorem 8.2 (Perceptron Convergence)**: If data is **linearly separable**, perceptron will find a separating hyperplane in **finite time**.

**Linearly separable** means there exists some line/hyperplane that perfectly separates the classes.

#### Two Scenarios:

1. **Separable data** → Perceptron converges! ✅
2. **Non-separable data** → Perceptron cycles forever ❌

For non-separable data, we need:
- Fixed iteration limit (early stopping)
- More sophisticated methods (kernels, neural networks, etc.)

### Why Linear Classifiers?

✅ **Simple** and interpretable  
✅ **Fast** to train  
✅ **Foundation** for more complex methods  
✅ Work well when classes are roughly linearly separable  

❌ **Limited** to linear decision boundaries  
❌ Fails on **XOR-like** problems

In [ ]:
@dataclass
class PerceptronModel:
    w: np.ndarray
    b: float

def perceptron_train(
    X: np.ndarray,
    y_pm1: np.ndarray,
    max_epochs: int = 50,
    shuffle: bool = True,
    seed: int = 0,
) -> Tuple[PerceptronModel, Dict[str, object]]:
    """Train a perceptron. y must be in {-1,+1}."""
    X = np.asarray(X, dtype=float)
    y = to_pm1(y_pm1).astype(int)
    n, d = X.shape
    rng = np.random.default_rng(seed)

    w = np.zeros(d, dtype=float)
    b = 0.0
    updates = 0

    for epoch in range(max_epochs):
        idx = np.arange(n)
        if shuffle:
            rng.shuffle(idx)
        mistakes = 0
        for i in idx:
            margin = y[i] * (X[i] @ w + b)
            if margin <= 0:
                w = w + y[i] * X[i]
                b = b + y[i]
                updates += 1
                mistakes += 1
        if mistakes == 0:
            return PerceptronModel(w=w, b=b), {"converged": True, "epochs": epoch+1, "updates": updates}
    return PerceptronModel(w=w, b=b), {"converged": False, "epochs": max_epochs, "updates": updates}

def perceptron_predict(model: PerceptronModel, X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    scores = X @ model.w + model.b
    yhat = np.where(scores >= 0, 1, -1).astype(int)
    return yhat


### 📊 Demo: Separable vs. Non-Separable Data

Let's see perceptron in action on two types of data:

1. **Linearly separable**: A line can perfectly separate the classes
   - Perceptron should converge (no mistakes after some iterations)

2. **Non-separable (ring/circle)**: Inner circle vs. outer ring
   - No single line can separate these
   - Perceptron will struggle and not converge

**What to observe**:
- Training risk (error rate)
- Number of epochs to convergence
- Whether the algorithm converges at all

In [ ]:
def make_linearly_separable(n: int = 200, seed: int = 0) -> Tuple[np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)
    X = rng.normal(size=(n, 2))
    true_w = np.array([1.2, -0.7])
    true_b = 0.1
    y = np.where(X @ true_w + true_b >= 0, 1, -1)
    return X, y

def make_ring_nonseparable(n: int = 300, seed: int = 0) -> Tuple[np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)
    angles = rng.uniform(0, 2*np.pi, size=n)
    radii = rng.uniform(0.0, 4.0, size=n)
    X = np.column_stack([radii*np.cos(angles), radii*np.sin(angles)])
    y = np.where(radii <= 1.0, 1, -1)  # inner disk vs outer region (not linearly separable)
    return X, y

def plot_2d_points(X: np.ndarray, y_pm1: np.ndarray, title: str = ""):
    y = to_pm1(y_pm1)
    plt.figure()
    plt.scatter(X[y==1, 0], X[y==1, 1], s=18, label="+1")
    plt.scatter(X[y==-1, 0], X[y==-1, 1], s=18, label="-1")
    plt.title(title)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.legend()
    plt.show()

X_sep, y_sep = make_linearly_separable()
plot_2d_points(X_sep, y_sep, "Linearly separable data")

model, info = perceptron_train(X_sep, y_sep, max_epochs=30, seed=0)
yhat = perceptron_predict(model, X_sep)
print(info, "train risk:", empirical_risk(y_sep, yhat))

X_ring, y_ring = make_ring_nonseparable()
plot_2d_points(X_ring, y_ring, "Non-separable (ring) data")

model2, info2 = perceptron_train(X_ring, y_ring, max_epochs=30, seed=0)
yhat2 = perceptron_predict(model2, X_ring)
print(info2, "train risk:", empirical_risk(y_ring, yhat2))


## 3. Kernelization: Escaping Linear Limitations 🔧

### The Problem: What if Data Isn't Linearly Separable?

Consider the "ring" problem we just saw:
```
          ○  ○  ○  ○
        ○   ●  ●   ○
       ○   ●    ●   ○
        ○   ●  ●   ○
          ○  ○  ○  ○
```

**No line** can separate the inner points (●) from outer points (○)!

### The Solution: Feature Mapping

**Key Idea**: Transform data to a **higher-dimensional space** where it becomes linearly separable!

#### Example: Polynomial Features

Original features: $(x_1, x_2)$  
Mapped features: $\phi(x) = (x_1, x_2, x_1^2, x_2^2, x_1x_2)$

Now in 5D space, the data might be linearly separable!

### The Kernel Trick 🎩✨

**Problem**: Computing $\phi(x)$ explicitly can be expensive (or even impossible for infinite dimensions).

**Kernel Trick**: We don't need $\phi(x)$ itself! We only need **inner products** $\phi(x) \cdot \phi(y)$.

A **kernel function** computes this directly:
$$k(x, y) = \phi(x) \cdot \phi(y)$$

**without** ever computing $\phi(x)$ explicitly!

### Common Kernels and Their Intuitions

#### 1. **Linear Kernel** 
$$k(x, y) = x^\top y$$
- Just the original inner product
- No transformation (baseline)

#### 2. **Polynomial Kernel**
$$k(x, y) = (\gamma x^\top y + r)^d$$
- Implicitly maps to polynomial features of degree $d$
- Captures interactions between features
- Example: $d=2$ captures all pairwise products

#### 3. **RBF (Gaussian) Kernel**
$$k(x, y) = \exp(-\gamma \|x - y\|^2)$$
- Measures "similarity" based on distance
- Close points → high similarity (≈1)
- Far points → low similarity (≈0)
- Maps to **infinite-dimensional** space!
- Most popular for non-linear data

#### 4. **RBF with L1 Distance**
$$k(x, y) = \exp(-\gamma \|x - y\|_1)$$
- Like RBF but uses Manhattan distance
- More robust to outliers in some cases

### Valid Kernels: The PSD Property

**Not every** function $k(x,y)$ is a valid kernel!

**Required property**: The kernel matrix $K_{ij} = k(x_i, x_j)$ must be **positive semi-definite (PSD)**.

**Why?** Because $K$ represents inner products, which always form PSD matrices.

We provide utilities to:
- Check if a matrix is PSD
- Construct explicit feature maps from kernel matrices (Lemma 8.3)

In [ ]:
# -----------------------------
# Common kernels (use whichever you need)
# -----------------------------

def kernel_linear(X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    return np.asarray(X) @ np.asarray(Y).T

def kernel_poly(X: np.ndarray, Y: np.ndarray, gamma: float = 1.0, r: float = 0.0, degree: int = 2) -> np.ndarray:
    return (gamma * (np.asarray(X) @ np.asarray(Y).T) + r) ** degree

def kernel_rbf_l2(X: np.ndarray, Y: np.ndarray, gamma: float = 1.0) -> np.ndarray:
    X = np.asarray(X); Y = np.asarray(Y)
    X2 = np.sum(X*X, axis=1, keepdims=True)
    Y2 = np.sum(Y*Y, axis=1, keepdims=True).T
    d2 = X2 + Y2 - 2*(X @ Y.T)
    return np.exp(-gamma * d2)

def kernel_rbf_l1(X: np.ndarray, Y: np.ndarray, gamma: float = 1.0) -> np.ndarray:
    X = np.asarray(X); Y = np.asarray(Y)
    K = np.empty((X.shape[0], Y.shape[0]), dtype=float)
    for i in range(X.shape[0]):
        K[i, :] = np.exp(-gamma * np.sum(np.abs(Y - X[i]), axis=1))
    return K

def is_psd(K: np.ndarray, tol: float = 1e-10) -> bool:
    """Check symmetric PSD: eigenvalues >= -tol."""
    K = np.asarray(K, dtype=float)
    if not np.allclose(K, K.T, atol=1e-8):
        return False
    eigs = np.linalg.eigvalsh(K)
    return bool(np.min(eigs) >= -tol)

def feature_map_from_gram(K: np.ndarray, tol: float = 1e-12) -> np.ndarray:
    """Lemma 8.3 constructive: if K is PSD, build B s.t. K = B B^T, rows are phi(x_i)."""
    K = np.asarray(K, dtype=float)
    if not np.allclose(K, K.T, atol=1e-8):
        raise ValueError("K must be symmetric.")
    eigvals, eigvecs = np.linalg.eigh(K)
    eigvals = np.maximum(eigvals, 0.0)
    keep = eigvals > tol
    B = eigvecs[:, keep] * np.sqrt(eigvals[keep])
    return B


In [ ]:
@dataclass
class KernelPerceptronModel:
    X_train: np.ndarray
    y_train: np.ndarray   # {-1,+1}
    alpha: np.ndarray     # integer-like update counts
    kernel: Callable[[np.ndarray, np.ndarray], np.ndarray]

def kernel_perceptron_train(
    X: np.ndarray,
    y_pm1: np.ndarray,
    kernel: Callable[[np.ndarray, np.ndarray], np.ndarray],
    max_epochs: int = 20,
    shuffle: bool = True,
    seed: int = 0,
) -> Tuple[KernelPerceptronModel, Dict[str, object]]:
    X = np.asarray(X, dtype=float)
    y = to_pm1(y_pm1).astype(int)
    n = X.shape[0]
    rng = np.random.default_rng(seed)
    alpha = np.zeros(n, dtype=float)

    K = kernel(X, X)
    updates = 0

    for epoch in range(max_epochs):
        idx = np.arange(n)
        if shuffle:
            rng.shuffle(idx)
        mistakes = 0
        for i in idx:
            # f(x_i) = sum_j alpha_j y_j k(x_j, x_i)
            score = np.sum(alpha * y * K[:, i])
            if y[i] * score <= 0:
                alpha[i] += 1.0
                updates += 1
                mistakes += 1
        if mistakes == 0:
            return KernelPerceptronModel(X, y, alpha, kernel), {"converged": True, "epochs": epoch+1, "updates": updates}
    return KernelPerceptronModel(X, y, alpha, kernel), {"converged": False, "epochs": max_epochs, "updates": updates}

def kernel_perceptron_predict(model: KernelPerceptronModel, X_new: np.ndarray) -> np.ndarray:
    X_new = np.asarray(X_new, dtype=float)
    K = model.kernel(model.X_train, X_new)  # shape (n_train, n_new)
    scores = (model.alpha * model.y_train)[:, None] * K
    scores = np.sum(scores, axis=0)
    return np.where(scores >= 0, 1, -1).astype(int)


### 🎯 Demo: Kernels Fix the Ring Problem!

Let's apply kernel perceptron to the ring data that linear perceptron failed on.

**What to expect**:
- **Linear kernel**: Should fail (equivalent to regular perceptron)
- **RBF kernel**: Should succeed! The transformation makes data separable

**Key Insight**: The RBF kernel measures **distances**, so it naturally separates inner (close to center) from outer (far from center) points.

This demonstrates the **power of kernelization**: same algorithm (perceptron), but now works on non-linear problems!

In [ ]:
# Linear kernel perceptron on ring
kp_lin, info_lin = kernel_perceptron_train(X_ring, y_ring, kernel=lambda A,B: kernel_linear(A,B), max_epochs=15, seed=0)
pred_lin = kernel_perceptron_predict(kp_lin, X_ring)
print("linear-kernel:", info_lin, "train risk:", empirical_risk(y_ring, pred_lin))

# RBF kernel perceptron on ring
kp_rbf, info_rbf = kernel_perceptron_train(X_ring, y_ring, kernel=lambda A,B: kernel_rbf_l2(A,B,gamma=1.0), max_epochs=15, seed=0)
pred_rbf = kernel_perceptron_predict(kp_rbf, X_ring)
print("rbf-kernel:", info_rbf, "train risk:", empirical_risk(y_ring, pred_rbf))


## 4. Empirical Risk Minimization (ERM): The Fundamental Principle ⚖️

### What is ERM?

**Empirical Risk Minimization** is the foundational principle of supervised learning:

$$\boxed{\hat{g}_n = \arg\min_{g \in \mathcal{M}} \hat{R}_n(g)}$$

**In words**: Among all classifiers in your model class $\mathcal{M}$, pick the one that makes the **fewest mistakes on training data**.

### Example Model Classes

- $\mathcal{M}_{\text{linear}}$ = all linear classifiers
- $\mathcal{M}_{\text{trees}}$ = all decision trees
- $\mathcal{M}_{\text{neural}}$ = all neural networks with given architecture

### The ERM Trap: Overfitting 🎣

**Crucial observation**: $\hat{R}_n(\hat{g}_n)$ is **optimistically biased**!

**Why?** Because we chose $\hat{g}_n$ to minimize training error. It's like:
- Training for an exam using **the exact test questions** 
- Of course you'll do well on those questions!
- But will you do well on **new** questions?

### The Bias-Complexity Trade-off

Consider two extremes:

1. **Very simple model** (e.g., constant predictor):
   - High training error ❌
   - BUT: Training error ≈ True error ✅
   
2. **Very complex model** (e.g., memorize all training points):
   - Zero training error ✅
   - BUT: Training error ≪ True error ❌ (overfitting!)

**Goal**: Find the sweet spot!

### How to Get Honest Error Estimates

#### Solution 1: Independent Test Set (Preferred!)

1. **Split** data: Training (for ERM) + Test (never seen)
2. **Train** on training set: $\hat{g}_n$
3. **Evaluate** on test set: $\hat{R}_{\text{test}}(\hat{g}_n)$

Now $\hat{R}_{\text{test}}$ is **unbiased** for $R(\hat{g}_n)$!

Plus, Hoeffding bound (from earlier) tells you how much to trust it:
$$\mathbb{P}(|\hat{R}_{\text{test}} - R| > \varepsilon) \leq 2e^{-2m\varepsilon^2}$$

#### Solution 2: VC Theory (Later!)

If you don't have a test set, VC theory gives **worst-case bounds** on generalization error based on:
- Training set size $n$
- Model complexity (VC dimension)

### Why Independent Test Data Matters

**KEY**: Test set must be:
- ✅ **Independently sampled** (same distribution as training)
- ✅ **Never used for training** (no peeking!)
- ✅ **Never used for model selection** (no tuning hyperparameters on it!)

If you violate any of these, your test error becomes biased!

In [ ]:
def train_test_split(X: np.ndarray, y: np.ndarray, test_size: float = 0.3, seed: int = 0):
    X = np.asarray(X); y = np.asarray(y)
    n = X.shape[0]
    rng = np.random.default_rng(seed)
    idx = np.arange(n); rng.shuffle(idx)
    m = int(round(n * test_size))
    test_idx = idx[:m]; train_idx = idx[m:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

# Example: evaluate perceptron with a test set + Hoeffding bound
Xtr, Xte, ytr, yte = train_test_split(X_sep, y_sep, test_size=0.3, seed=1)
model, info = perceptron_train(Xtr, ytr, max_epochs=30, seed=0)
pred_te = perceptron_predict(model, Xte)
risk_te = empirical_risk(yte, pred_te)

eps = 0.05
print("test risk:", risk_te)
print("P(|Rhat - R| > eps) <=", hoehffding_test_bound(len(yte), eps))


## 4.2 Precision, Recall, and the Confusion Matrix 📊

### Beyond Accuracy: When Classes are Imbalanced

**Accuracy** = (# correct) / (# total) is not always enough!

**Example**: Cancer detection
- 99% of patients are healthy
- A classifier that always predicts "healthy" has 99% accuracy!
- But it **never detects cancer** 😱

We need more nuanced metrics.

### The Confusion Matrix

For binary classification (positive/negative):

|                    | **Predicted +** | **Predicted -** |
|--------------------|-----------------|-----------------|
| **Actual +**       | TP (True Pos)   | FN (False Neg)  |
| **Actual -**       | FP (False Pos)  | TN (True Neg)   |

### Key Metrics

#### 1. **Precision** (How many predicted positives are actually positive?)
$$\text{Precision} = \frac{TP}{TP + FP} = P(Y=1 \mid \hat{Y}=1)$$

**Intuition**: "When I say positive, how often am I right?"

#### 2. **Recall** (How many actual positives did I find?)
$$\text{Recall} = \frac{TP}{TP + FN} = P(\hat{Y}=1 \mid Y=1)$$

**Intuition**: "Of all positives, how many did I catch?"  
Also called **Sensitivity** or **True Positive Rate**.

#### 3. **F1-Score** (Harmonic mean of precision and recall)
$$F1 = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$

**Intuition**: Balances precision and recall into a single number.

### The Precision-Recall Trade-off

Usually, there's a trade-off:
- **High recall** (catch all positives) → many false alarms → low precision
- **High precision** (only sure bets) → miss many positives → low recall

Example:
- Spam filter with high recall: catches all spam but some real emails too
- Spam filter with high precision: never wrong but misses some spam

### Edge Cases to Watch Out For ⚠️

1. **No predicted positives** ($TP + FP = 0$):
   - Precision = undefined (NaN)
   - Often happens with very imbalanced data

2. **No actual positives** ($TP + FN = 0$):
   - Recall = undefined (NaN)
   - Rare unless data is severely biased

Our utilities handle these cases gracefully by returning NaN.

In [ ]:
def metrics_on_testset(y_true, y_pred, label_space: str = "pm1") -> Dict[str, float]:
    """label_space: 'pm1' for {-1,1}, '01' for {0,1}."""
    if label_space == "pm1":
        y_true01 = to_01(to_pm1(y_true))
        y_pred01 = to_01(to_pm1(y_pred))
    elif label_space == "01":
        y_true01 = np.asarray(y_true).astype(int)
        y_pred01 = np.asarray(y_pred).astype(int)
    else:
        raise ValueError("label_space must be 'pm1' or '01'")
    return precision_recall_f1(y_true01, y_pred01)

# Edge-case demos
y_true01 = np.array([1,0,0,0,0,0,0,0,0,0])
always1 = np.ones_like(y_true01)
always0 = np.zeros_like(y_true01)
print("always 1:", metrics_on_testset(y_true01, always1, label_space="01"))
print("always 0:", metrics_on_testset(y_true01, always0, label_space="01"))


## 4.3 Finite Hyperplane ERM (Theorem 8.16) 🎲

### Motivation: Can We Get Generalization Guarantees?

We've seen:
- Training error $\hat{R}_n$ can be optimistically biased
- Test sets help, but require holding out data

**Question**: Can we get **theoretical guarantees** about generalization?

**Answer**: Yes! But only for **finite model classes**.

### The Setup: Finite Set of Hyperplanes

Consider a **restricted model class** $\mathcal{M}_n$ containing only hyperplanes that pass through **$m$ training points** (where $m$ is the dimension).

**Key property**: This class is **finite**!
- There are $\binom{n}{m}$ ways to choose $m$ points
- Each defines 2 hyperplanes (one for each side)
- Total: $|\mathcal{M}_n| = 2\binom{n}{m}$ classifiers

### Theorem 8.16: Generalization Bound

For this finite class, with high probability:

$$R(\hat{g}) \leq \inf_{g \in \mathcal{M}} R(g) + \varepsilon$$

with probability at least:

$$1 - \exp(2m\varepsilon) \cdot (2\binom{n}{m} + 1) \cdot \exp(-n\varepsilon^2/2)$$

**What this means**:
- Our ERM classifier $\hat{g}$ is nearly as good as the **best in class**
- The bound gets tighter with more data ($n$ large)
- But it degrades with model complexity ($\binom{n}{m}$ large)

### Practical Implementation

In practice, $\binom{n}{m}$ is huge! For $n=100, m=3$: $\binom{100}{3} = 161,700$.

**Solution**: **Random sampling approach**
1. Randomly sample many $m$-tuples of points
2. Build hyperplane through each
3. Pick the one with lowest training error

This approximates ERM over the finite class and is computationally feasible!

### Why This Matters

This is our **first rigorous generalization guarantee**! It shows:
- Finite model classes → predictable generalization
- But $\binom{n}{m}$ grows fast → loose bounds
- **Foreshadowing**: Need better tools for infinite/large classes → VC theory!

In [ ]:
def hyperplane_from_d_points(points: np.ndarray, tol: float = 1e-12) -> Tuple[np.ndarray, float]:
    """
    Build a hyperplane a^T x + b = 0 passing through d points in R^d.
    Returns (a, b) up to scaling.
    Uses nullspace of [points, 1].
    """
    P = np.asarray(points, dtype=float)
    d = P.shape[1]
    if P.shape[0] != d:
        raise ValueError("Need exactly d points in R^d to determine a hyperplane uniquely (up to scale).")
    A = np.hstack([P, np.ones((d, 1))])  # shape (d, d+1)
    # Find vector v in nullspace of A (A v = 0). Nullspace dimension should be 1 in general position.
    U, S, Vt = np.linalg.svd(A)
    v = Vt[-1, :]  # smallest singular value vector
    if np.linalg.norm(A @ v) > 1e-8:
        raise RuntimeError("Failed to find a stable nullspace vector; points may be degenerate.")
    a = v[:d]
    b = v[d]
    # normalize for stability
    norm = np.linalg.norm(a)
    if norm < tol:
        raise RuntimeError("Degenerate hyperplane (normal ~ 0).")
    a = a / norm
    b = b / norm
    return a, b

def predict_hyperplane(a: np.ndarray, b: float, X: np.ndarray) -> np.ndarray:
    scores = np.asarray(X, dtype=float) @ a + b
    return np.where(scores >= 0, 1, 0).astype(int)  # {0,1} decision rule

def sampled_hyperplane_erm(
    X: np.ndarray,
    y01: np.ndarray,
    n_candidates: int = 2000,
    seed: int = 0,
) -> Dict[str, object]:
    """Sample many d-tuples, build hyperplanes, pick best empirical-risk rule."""
    X = np.asarray(X, dtype=float)
    y01 = np.asarray(y01).astype(int)
    n, d = X.shape
    if n < d:
        raise ValueError("Need n >= d to sample d points.")
    rng = np.random.default_rng(seed)

    best = {"risk": float("inf"), "a": None, "b": None, "flip": 0}
    for _ in range(n_candidates):
        idx = rng.choice(n, size=d, replace=False)
        try:
            a, b = hyperplane_from_d_points(X[idx, :])
        except Exception:
            continue
        pred = predict_hyperplane(a, b, X)
        r = empirical_risk(y01, pred)
        # also consider the opposite side (phi+ vs phi-)
        pred_flip = 1 - pred
        r2 = empirical_risk(y01, pred_flip)
        if r2 < r:
            r, pred = r2, pred_flip
            flip = 1
        else:
            flip = 0
        if r < best["risk"]:
            best = {"risk": float(r), "a": a, "b": float(b), "flip": flip}
    return best


In [ ]:
# Demo in 2D (d=2): sample hyperplane ERM on separable data (convert labels to {0,1})
best = sampled_hyperplane_erm(X_sep, to_01(y_sep), n_candidates=3000, seed=0)
best["risk"], best["a"], best["b"], best["flip"]


### 📐 Computing Theorem 8.16 Bound

The bound from Theorem 8.16 is:

$$\mathbb{P}\left\{R(\hat{\phi}) > \inf_{\mathcal{M}} R(\phi) + \varepsilon\right\} \leq \exp(2m\varepsilon) \cdot (2\binom{n}{m} + 1) \cdot \exp\left(-\frac{n\varepsilon^2}{2}\right)$$

**Valid when**: $\frac{2m}{n} \leq \varepsilon \leq 1$

**Components**:
- $\exp(2m\varepsilon)$: Penalty for dimension
- $2\binom{n}{m} + 1$: Size of model class
- $\exp(-n\varepsilon^2/2)$: Concentration benefit from data

**Interpretation**: 
- More data ($n$ ↑) → bound decreases (good!)
- More dimension ($m$ ↑) → bound increases (bad!)
- Larger tolerance ($\varepsilon$ ↑) → bound increases

Below we compute this bound numerically.

In [ ]:
def comb(n: int, k: int) -> int:
    return math.comb(n, k)

def theorem_816_bound(n: int, m_dim: int, eps: float) -> float:
    if not (0 < eps <= 1):
        raise ValueError("eps in (0,1]")
    # exp(2 m eps) * (2*C(n,m)+1) * exp(-n eps^2 / 2)
    return float(math.exp(2*m_dim*eps) * (2*comb(n, m_dim) + 1) * math.exp(-n*(eps**2)/2))

# Example numbers (small just for illustration)
print(theorem_816_bound(n=200, m_dim=2, eps=0.2))


## 5. VC Theory: The Mathematics of Generalization 🎓

### The Big Question

**How do we get generalization bounds for infinite model classes?**

Examples of infinite classes:
- All linear classifiers (infinite ways to set $w$ and $b$)
- All polynomials of degree $\leq d$
- All neural networks with given architecture

Theorem 8.16 only worked for **finite** classes. We need a new approach!

### The Key Insight: Effective Complexity

**VC theory's brilliant idea**: Even though a model class is infinite, on **any finite dataset**, it can only produce **finitely many** different classification patterns.

### Shattering: Measuring Expressiveness

**Definition**: A set of $n$ points is **shattered** by class $\mathcal{A}$ if $\mathcal{A}$ can produce **all $2^n$ possible labelings** of those points.

#### Example: Linear Classifiers in 2D

**Can we shatter 3 points?** 

```
     ●  ●  ●
```

Try all $2^3 = 8$ labelings:
- $(+,+,+)$, $(+,+,-)$, $(+,-,+)$, $(-,+,+)$, ...

**Answer**: YES! For any 3 points in general position, some line achieves each labeling.

**Can we shatter 4 points?**

**Answer**: NO! Some labelings (like XOR: $(+,-,+,-)$) cannot be achieved by any line.

### The Shattering Number

**Definition**: $s(\mathcal{A}, n)$ = maximum number of **distinct labelings** $\mathcal{A}$ can produce on **any** set of $n$ points.

**Properties**:
- $s(\mathcal{A}, n) \leq 2^n$ (can't have more labelings than $2^n$)
- If $\mathcal{A}$ can shatter some $n$ points, then $s(\mathcal{A}, n) = 2^n$
- Usually grows like polynomial after some point (not exponential!)

### The VC Dimension

**Definition**: The **VC dimension** $V_{\mathcal{A}}$ is the **largest** $n$ such that $\mathcal{A}$ can shatter $n$ points.

**Examples**:
- Linear classifiers in $\mathbb{R}^d$: $V \approx d + 1$ (Lemma 8.38)
- Polynomial classifiers degree $d$ in $\mathbb{R}$: $V = d + 1$
- Neural networks: depends on architecture (typically $\approx$ number of parameters)

### Why VC Dimension Matters

**VC dimension** = **effective complexity** of model class

- **Low VC dim**: Simple models, good generalization, but limited expressiveness
- **High VC dim**: Complex models, can fit complex patterns, but risk overfitting

### The Sauer-Shelah Lemma: The Key Bound

**Theorem (Sauer-Shelah)**: If $V_{\mathcal{A}} = V$, then:

$$s(\mathcal{A}, n) \leq \sum_{i=0}^{V-1} \binom{n}{i} \leq \left(\frac{en}{V}\right)^V$$

**What this means**:
- Even for infinite $\mathcal{A}$, shattering number grows **polynomially** (not exponentially!)
- The polynomial degree is the **VC dimension**
- This replaces $2\binom{n}{m}$ from Theorem 8.16

### VC Generalization Bounds

**Theorem 8.29** (Uniform convergence): For any $\mathcal{A}$ with shattering number $s(\mathcal{A},n)$:

$$\mathbb{P}\left(\sup_{A \in \mathcal{A}} |\nu_n(A) - \nu(A)| > \varepsilon\right) \leq 8 \cdot s(\mathcal{A}, n) \cdot e^{-n\varepsilon^2/32}$$

(valid when $n\varepsilon^2 \geq 2$)

**Corollary 8.31** (For classifiers): Adjusted constants for 0-1 loss:

$$\mathbb{P}\left(\sup_{g \in \mathcal{M}} |R(g) - \hat{R}_n(g)| > \varepsilon\right) \leq 8 \cdot s(\mathcal{M}, n) \cdot e^{-n\varepsilon^2/64}$$

### The Complete Picture

For ERM classifier $\hat{g}_n$:

$$\boxed{R(\hat{g}_n) \leq \underbrace{\hat{R}_n(\hat{g}_n)}_{\text{training error}} + \underbrace{\varepsilon}_{\text{generalization gap}}}$$

with high probability (controlled by VC bounds).

**Key takeaways**:
1. ✅ Works for **infinite** model classes!
2. ✅ Only requires **VC dimension** (no need to count classifiers)
3. ✅ Explains the **bias-variance trade-off** mathematically
4. ⚠️ Bounds are often loose in practice (but theoretically tight!)

In [ ]:
def sauer_shelah_upper(n: int, vc_dim: int) -> int:
    """Upper bound on shattering number via Sauer–Shelah: sum_{i=0}^{V-1} C(n,i)."""
    if vc_dim <= 0:
        return 1
    upper = 0
    for i in range(0, min(vc_dim, n+1)):
        upper += math.comb(n, i)
    return upper

def lemma_837_poly_upper(N: int, k: int) -> float:
    """Lemma 8.37 style: sum_{i=0}^{k-1} C(N,i) <= (eN/k)^k."""
    if k <= 0:
        return 1.0
    return float((math.e * N / k) ** k)

def vc_generalization_bound_theorem_829(n: int, eps: float, shattering_number: float) -> float:
    if eps <= 0:
        raise ValueError("eps>0")
    return float(8.0 * shattering_number * math.exp(-n * eps * eps / 32.0))

def vc_generalization_bound_cor_831(n: int, eps: float, shattering_number: float) -> float:
    if eps <= 0:
        raise ValueError("eps>0")
    return float(8.0 * shattering_number * math.exp(-n * eps * eps / 64.0))

# Example: linear classifiers in R^m have VC-dim about m+1 (Lemma 8.38 informal)
def bound_linear_classifiers(n: int, eps: float, m_dim: int) -> Dict[str, float]:
    vc_dim = m_dim + 1
    s_upper = sauer_shelah_upper(n, vc_dim)
    return {
        "vc_dim≈m+1": vc_dim,
        "sauer_shelah_upper": float(s_upper),
        "Thm8.29_bound": vc_generalization_bound_theorem_829(n, eps, s_upper),
        "Cor8.31_bound": vc_generalization_bound_cor_831(n, eps, s_upper),
    }

bound_linear_classifiers(n=200, eps=0.1, m_dim=2)


## 6. Martingale Concentration: ERM Training Risk (Theorem 8.43) 🎲

### A Different Question

So far we asked: *How well does training error estimate true error?*

Now we ask: *How concentrated is the training error itself?*

### The Setup

Consider the **ERM classifier** $\hat{g}_n$ (depends on random training data).

The **training risk** $\hat{R}_n(\hat{g}_n)$ is itself a **random variable**!

### Theorem 8.43: Azuma-Hoeffding Concentration

For the ERM training risk:

$$\mathbb{P}\left(|\hat{R}_n(\hat{g}_n) - \mathbb{E}[\hat{R}_n(\hat{g}_n)]| > \varepsilon\right) < 2e^{-n\varepsilon^2/2}$$

**What this means**:
- Training error concentrates around its **expectation**
- With $n$ examples, deviation larger than $\varepsilon$ is exponentially unlikely
- This is a **martingale concentration** result (Azuma's inequality)

### Why This is Different

**Previous results** (Theorems 8.29, 8.31): 
- About gap between $R(g)$ and $\hat{R}_n(g)$ 
- Need VC dimension / shattering numbers

**This result** (Theorem 8.43):
- About concentration of $\hat{R}_n(\hat{g}_n)$ around its mean
- True for **any** ERM procedure (no VC dimension needed!)
- Uses martingale theory (bounded differences)

### The Intuition: Martingale Argument

Think of revealing training examples one by one:
- After seeing $i$ examples, we have partial information
- Adding one more example changes training error by **at most $1/n$**
- This "bounded difference" property → concentration via Azuma

### Practical Implication

This bound tells you how **stable** the ERM training error is:
- If you resample training data, training error won't change much
- Useful for understanding **variance** of the learning procedure

In [ ]:
def erm_training_risk_concentration_bound(n: int, eps: float) -> float:
    """Thm 8.43: 2 exp(-n eps^2 / 2)."""
    if n <= 0 or eps <= 0:
        raise ValueError("n>0, eps>0.")
    return float(2.0 * math.exp(-n * eps * eps / 2.0))

print(erm_training_risk_concentration_bound(500, 0.05))


## 🎓 Chapter Summary: Putting It All Together

### The Big Picture

We've journeyed through the foundations of **supervised learning theory**. Here's how everything connects:

#### 1️⃣ **The Goal**: Learn from Data
- **Input**: Training examples $(X_i, Y_i)$
- **Output**: Classifier $g(X)$ that predicts well on new data
- **Measure**: Risk $R(g) = \mathbb{P}(Y \neq g(X))$

#### 2️⃣ **The Method**: Empirical Risk Minimization
- **Can't compute** true risk $R(g)$ (don't know true distribution)
- **Can compute** empirical risk $\hat{R}_n(g)$ (average on training data)
- **ERM principle**: Pick $\hat{g} = \arg\min_{g \in \mathcal{M}} \hat{R}_n(g)$

#### 3️⃣ **The Challenge**: Generalization Gap
- Training error $\hat{R}_n(\hat{g})$ is **optimistically biased** ⚠️
- True error $R(\hat{g})$ is what we really care about
- **Gap**: $R(\hat{g}) - \hat{R}_n(\hat{g})$ depends on:
  - Model complexity (VC dimension)
  - Training set size
  - Test set independence

#### 4️⃣ **The Solutions**: 

**For Simple Cases**:
- ✅ **Independent test set** + Hoeffding bound
  - Most practical approach
  - Requires holding out data
  
**For Finite Model Classes**:
- ✅ **Theorem 8.16**: Bounds using $|\mathcal{M}|$
  - Works but requires finite class
  
**For Infinite Classes**:
- ✅ **VC Theory** (Theorems 8.29, 8.31)
  - Replace size $|\mathcal{M}|$ with shattering number $s(\mathcal{M}, n)$
  - Bounded by Sauer-Shelah: $s(\mathcal{M}, n) \leq (en/V)^V$
  - VC dimension $V$ measures **effective complexity**

**For Training Stability**:
- ✅ **Theorem 8.43**: Martingale concentration
  - Training error concentrates around its mean
  - Independent of model complexity

### 🔑 Key Insights

1. **Bias-Variance Trade-off**:
   - Simple models: high bias, low variance → underfitting
   - Complex models: low bias, high variance → overfitting
   - Sweet spot: minimize $\text{bias}^2 + \text{variance}$

2. **Sample Complexity**:
   - Need $n \gtrsim V/\varepsilon^2$ examples for $\varepsilon$-accurate learning
   - More complex models (high $V$) need more data!

3. **The Kernel Trick**:
   - Transforms non-linear problems → linear in feature space
   - Never need explicit features, just kernel function
   - Enables infinite-dimensional feature spaces!

4. **Perceptron Theorem**:
   - Converges if data is linearly separable
   - Number of updates bounded by margin
   - Foundation for SVMs and neural networks

### 📊 Practical Takeaways

**When building a classifier**:
1. ✅ Always use a separate test set (never train on it!)
2. ✅ Check if data is linearly separable (try linear first)
3. ✅ Use kernels for non-linear problems (RBF is popular)
4. ✅ Monitor training vs. test error (detect overfitting)
5. ✅ Consider model complexity vs. data size (VC dimension)

**When evaluating**:
1. ✅ Don't trust training error alone
2. ✅ Use precision/recall for imbalanced data
3. ✅ Use Hoeffding bounds to quantify test set reliability
4. ✅ Consider worst-case VC bounds for safety-critical applications

### 🎯 What You've Mastered

- ✅ **Foundation**: Loss functions, risk, empirical risk
- ✅ **Algorithms**: Perceptron, kernel perceptron
- ✅ **Theory**: ERM principle, generalization bounds
- ✅ **VC Theory**: Shattering, VC dimension, Sauer-Shelah
- ✅ **Concentration**: Hoeffding, Azuma-Hoeffding
- ✅ **Practice**: Train/test splits, kernels, evaluation metrics

---

## ✅ Function Index (Quick Reference for Problem Solving)

Copy these functions when solving exercises or exam problems!

### 📐 Core Risk & Metrics
```python
empirical_risk(y_true, y_pred)              # Compute 0-1 loss rate
precision_recall_f1(y_true01, y_pred01)     # Full metric suite
metrics_on_testset(y_true, y_pred, label_space="pm1")
train_test_split(X, y, test_size=0.3, seed=0)
```

### 📊 Statistical Bounds
```python
hoehffding_test_bound(m, eps)               # P(|R_test - R| > eps)
required_test_size_for_eps_delta(eps, delta) # Inverse: m needed
erm_training_risk_concentration_bound(n, eps) # Thm 8.43
```

### 🧠 Linear Classifiers
```python
perceptron_train(X, y_pm1, max_epochs=50, seed=0)
perceptron_predict(model, X)
```

### 🔧 Kernels & Kernel Perceptron
```python
# Kernel functions
kernel_linear(X, Y)
kernel_poly(X, Y, gamma=1.0, r=0.0, degree=2)
kernel_rbf_l2(X, Y, gamma=1.0)
kernel_rbf_l1(X, Y, gamma=1.0)

# Utilities
is_psd(K, tol=1e-10)                        # Check valid kernel
feature_map_from_gram(K)                     # Lemma 8.3 construction

# Training
kernel_perceptron_train(X, y_pm1, kernel, max_epochs=20, seed=0)
kernel_perceptron_predict(model, X_new)
```

### 🎲 Finite Hyperplane ERM (Thm 8.16)
```python
hyperplane_from_d_points(points)             # Build hyperplane
sampled_hyperplane_erm(X, y01, n_candidates=2000, seed=0)
theorem_816_bound(n, m_dim, eps)             # Compute Thm 8.16 bound
```

### 📚 VC Theory Bounds
```python
sauer_shelah_upper(n, vc_dim)                # Upper bound on s(M,n)
lemma_837_poly_upper(N, k)                   # (eN/k)^k bound
vc_generalization_bound_theorem_829(n, eps, s)
vc_generalization_bound_cor_831(n, eps, s)
bound_linear_classifiers(n, eps, m_dim)      # All-in-one for linear
```

### 🎯 Helper Functions
```python
to_pm1(y)          # Convert {0,1} → {-1,+1}
to_01(y_pm1)       # Convert {-1,+1} → {0,1}
zero_one_loss(y_true, y_pred)
confusion_matrix_01(y_true01, y_pred01)
```

---

### 🚀 Ready for More?

You now have a solid foundation in:
- **Pattern recognition algorithms** (perceptron, kernels)
- **Statistical learning theory** (VC dimension, generalization)
- **Practical machine learning** (train/test, evaluation)

**Next steps**:
- Chapter 10: High-dimensional statistics
- Chapter 11: Dimensionality reduction (PCA, manifold learning)
- Advanced topics: Neural networks, ensemble methods, optimization

**Good luck on your exams and projects!** 📚✨